# About: ECインスタンスの作成

---

Moodle環境を構築するための計算資源をAWS EC2インスタンスで作成します。

## 概要

### 前提条件

このNotebookで EC2インスタンスを作成する際の前提条件を以下に示します。

* AWSを操作するための[認証情報（アクセスキー、シークレットキー）](https://docs.aws.amazon.com/ja_jp/IAM/latest/UserGuide/id_credentials_access-keys.html#Using_CreateAccessKey)があること
* [Amazon VPC](https://docs.aws.amazon.com/ja_jp/vpc/latest/userguide/what-is-amazon-vpc.html)と[サブネット](https://docs.aws.amazon.com/ja_jp/vpc/latest/userguide/VPC_Subnets.html)が作成済であること
* [EC2キーペア](https://docs.aws.amazon.com/ja_jp/AWSEC2/latest/UserGuide/ec2-key-pairs.html)が作成済であること
* 起動したEC2インスタンスに対して、このNotebook環境からsshでログイン可能であること
    - ネットワーク的に到達可能なこと
    - ログインするためのSSH鍵ファイルをこのNotebook環境に配置してあること

### 準備

AWS EC2インスタンスを操作するために必要となるライブラリをインストールします。

このNotebookでは Ansible を利用してEC2インスタンスを作成します。
現在の環境にインストールされているライブラリとの競合をさけるためにvenvで独立した環境を構築します。

まず venv で仮想環境を作成します。

In [ ]:
!python -mvenv .venv

venvの環境に ansible とAWSの操作に必要となるライブラリをインストールします。

In [ ]:
!.venv/bin/pip install ansible-core
!.venv/bin/ansible-galaxy collection install -f amazon.aws:10.1.0
!.venv/bin/pip install -r $HOME/.ansible/collections/ansible_collections/amazon/aws/requirements.txt

## パラメータの設定

このNotebookで作成するAWS EC2インスタンスのパラメータを設定します。

### AWSの認証情報

EC2インスタンスの操作を行う際に必要となるAWSの認証情報を指定します。

AWSの認証情報（アクセスキー、シークレットキー）を取得する手順については[「アクセスキーの管理」](https://docs.aws.amazon.com/ja_jp/IAM/latest/UserGuide/id_credentials_access-keys.html#Using_CreateAccessKey)などを参照してください。

次のセルを実行すると入力枠が表示されるのでAWSのアクセスキーの値を入力してください。

> 入力後に Enter キーを押すことで入力が完了します。

In [ ]:
from getpass import getpass
aws_access_key = getpass()

AWSのシークレットキーを入力してください。

In [ ]:
aws_secret_key = getpass()

このNotebookから実行するコマンドでAWS認証情報を参照できるようにするために、環境変数`AWS_ACCESS_KEY_ID`, `AWS_SECRET_ACCESS_KEY`の設定を行います。

In [ ]:
import os

os.environ['AWS_ACCESS_KEY_ID'] = aws_access_key
os.environ['AWS_SECRET_ACCESS_KEY'] = aws_secret_key

### EC2インスタンスの名前の指定

EC2インスタンスの名前を指定してください。指定した値はEC2インスタンスのNameタグに設定されます。

In [ ]:
# (例)
# aws_ec2_name = 'moodle'

aws_ec2_name = 

### ECインスタンスを起動する環境の指定

EC2インスタンスを起動するリージョンを指定してください。

In [ ]:
# (例)
# aws_region = 'ap-northeast-1' # アジアパシフィック (東京)

aws_region = 

EC2インスタンスのサブネットIDを指定してください。

In [ ]:
# (例)
# aws_vpc_subnet_id = 'subnet-xxxxxxxxxxxxxxxxx'

aws_vpc_subnet_id = 

### EC2インスタンスに割り当てるリソースの指定

起動するEC2インスタンスに割り当てるリソース量を指定します。

EC2インスタンスの[インスタンスタイプ](https://aws.amazon.com/jp/ec2/instance-types/)を指定してください。

In [ ]:
# (例)
# aws_instance_type = 'm7i.large'
# aws_instance_type = 't3a.large'

aws_instance_type = 

EC2インスタンスのルートボリュームサイズ(GB)を指定してください。

In [ ]:
# (例)
# aws_volume_size = 32

aws_volume_size = 

### AMIの指定

ECインスタンスの[AMI](https://docs.aws.amazon.com/ja_jp/AWSEC2/latest/UserGuide/AMIs.html)を指定します。このアプリケーションテンプレートでは、ノードのOSとしてRockyLinux9を使用します。


指定したリージョンに対応するAMI IDを[Rocky Linux 9](https://aws.amazon.com/marketplace/pp/prodview-ygp66mwgbl2ii)で確認して次のセルに指定してください。

In [ ]:
# (例)
# aws_image_id = 'ami-0bdd4b6c4c6feca84 # RockyLinux 9.6 x86_64, Asia Pacific (Tokyo)

aws_image_id  = 'ami-0bdd4b6c4c6feca84'

### セキュリティグループの指定

[セキュリティグループ](https://docs.aws.amazon.com/ja_jp/AWSEC2/latest/UserGuide/ec2-security-groups.html)を指定します。

次のセルでセキュリティグループIDを指定してください。

In [ ]:
# (例)
# aws_security_group = 'sg-xxxxxxxxxxxxxxxxx'

aws_security_group = 

### キーペアの指定

EC2インスタンスに設定する[キーペア](https://docs.aws.amazon.com/ja_jp/AWSEC2/latest/UserGuide/ec2-key-pairs.html)を指定します。

In [ ]:
# (例)
# aws_key_name = 'key_name'

aws_key_name = 

## EC2インスタンスの作成

これまでに入力したパラメータを指定したEC2インスタンスを作成します。

### Ansible Playbook の生成

このNotebookではEC2インスタンスを作成するために [Ansible](https://www.ansible.com/)を利用します。

次のセルを実行するとEC2インスタンスを作成するための [Ansible Playbook](https://docs.ansible.com/ansible/latest/user_guide/playbooks_intro.html) を生成します。

In [ ]:
%run scripts/edit_conf.py

playbook = create_conf_file('localhost', 'aws-ec2.yml')
with playbook.open(mode='w') as f:
    f.write(f'''
- hosts: localhost
  tasks:
  - amazon.aws.ec2_instance:
      name: {aws_ec2_name}
      image_id: {aws_image_id}
      instance_type: {aws_instance_type}
      vpc_subnet_id: {aws_vpc_subnet_id}
      security_group: {aws_security_group}
      region: {aws_region}
      key_name: {aws_key_name}
      volumes:
      - device_name: /dev/sda1
        ebs:
          volume_size: {aws_volume_size}
          delete_on_termination: true
''')
generate_edit_link(playbook)

上のリンクをクリックするとブラウザの新しいウィンドウ（タブ）で playbook の編集画面が開きます。編集を行った場合は `ctrl-s` またはメニューの[File]-[Save]で編集結果を保存してください。

EC2インスタンスを作成するためのパラメータの詳細についてはAnsibleのドキュメント[「amazon.aws.ec2_instance モジュール」](https://docs.ansible.com/ansible/latest/collections/amazon/aws/ec2_instance_module.html)を参照してください。
例えばEC2インスタンスのプライベートIPアドレスに `172.30.2.10` を設定するには以下のような指定を追加してください。

```
      network:
        private_ip_address: 172.30.2.10
```

確認のためplaybookの内容を表示します。

In [ ]:
!cat {playbook}

### EC2インスタンスの起動

playbookを実行してEC2インスタンスを起動します。

In [ ]:
!.venv/bin/ansible-playbook -c local {playbook}

作成したEC2インスタンスのIDなどを確認するために[amazon.aws.ec2_instance_infoモジュール](https://docs.ansible.com/ansible/latest/collections/amazon/aws/ec2_instance_info_module.html)を用いて情報を取得します。

In [ ]:
import json
out = !.venv/bin/ansible localhost -c local -m amazon.aws.ec2_instance_info -a 'region={aws_region} \
    filters="{{"tag:Name": "{aws_ec2_name}", "instance-state-name": "running", "network-interface.subnet-id": "{aws_vpc_subnet_id}"}}"'
idx = [i for i, x in enumerate(out) if x.startswith('localhost |')][0]
if out[idx] == 'localhost | SUCCESS => {':
    ec2_info = json.loads(' '.join(['{'] + out[(idx + 1):]))
    for line in out[idx:]:
        print(line)
else:
    for line in out:
        print(line)
    raise RuntimeError("error!")

取得した情報からインスタンスIDの一覧を表示します。

In [ ]:
instance_ids = [x['instance_id'] for x in ec2_info['instances']]
for id in instance_ids:
    print(id)

取得した情報はインスタンスID以外の情報も多く含まれています。
例えばプライベートIPアドレスを取得するには次のセルに示した手順で一覧を表示できます。

In [ ]:
for x in ec2_info['instances']:
    print(x['private_ip_address'])

取得した情報の内容については[Ansibleのドキュメント](https://docs.ansible.com/ansible/latest/collections/amazon/aws/ec2_instance_info_module.html#return-values)を参照してください。

## Ansibleの設定

起動したEC2インスタンスをAnsibleで操作するための設定を行います。

### パラメータの設定

EC2インスタンスを登録するAnsibleのグループ名を指定してください。

In [ ]:
# (例)
# target_group = 'Moodle'
# target_group = aws_ec2_name  # EC2インスタンスのNameと同じグループ名にする場合

target_group = aws_ec2_name

このNotebook環境からEC2インスタンスに接続する際のホスト名（IPアドレス）を指定してください。

In [ ]:
# (例)
# target_hostname = 'moodle.example.org'
# target_hostname = '172.30.1.10'


target_hostname = 

EC2インスタンスにログインするためのSSH秘密鍵のファイルを指定してください。

In [ ]:
# (例)
# ssh_private_key_file = '~/.ssh/aws_key.pem'
# ssh_private_key_file = '~/.ssh/id_rsa'

ssh_private_key_file = 

### インベントリの作成

Ansibleのインベントリ`inventory.yml`をカレントディレクトリに作成します。

In [ ]:
%run scripts/group.py

inventory = {'all': {'children': {
    target_group: {
        'hosts': {
            target_hostname: {},
        },
        'vars': {
            'ansible_user': 'rocky',
            'ansible_ssh_private_key_file': os.path.expanduser(ssh_private_key_file),
        }
    }
}}}

generate_edit_link(update_inventory_yml(inventory))

次のセルを実行すると作成したインベントリの内容を表示します。インベントリの内容を変更したい場合は、上のセルの出力結果に表示しているリンクから編集することができます。

In [ ]:
!cat inventory.yml

### ansible.cfg の作成

先程、カレントディレクトリに作成した`inventory.yml`をAnsibleのインベントリとして指定するための設定を行います。

> カレントディレクトリにコンフィギュレーションファイル`ansible.cfg`を作成すると、Ansibleを実行する際にその設定が適用されます。

In [ ]:
cfg = setup_ansible_cfg()
generate_edit_link(cfg)

次のセルを実行すると作成したコンフィギュレーションファイルの内容を表示します。コンフィギュレーションファイルの内容を変更したい場合は、上のセルの出力結果に表示しているリンクから編集することができます。

In [ ]:
!cat ansible.cfg

### EC2インスタンスへの接続確認

EC2インスタンスに対して Ansible で接続できることを確認します。

In [ ]:
!ssh-keygen -R {target_hostname} || true
!mkdir -p -m 0700 ~/.ansible/cp
!env ANSIBLE_HOST_KEY_CHECKING=False \
    ansible {target_group} -m ping

正常に接続できると以下のように表示されます。

```
XXX.XXX.XXX.XXX | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
```

`~/.ssh/known_hosts`の内容を更新します。

In [ ]:
!ssh-keyscan -H {target_hostname} >> ~/.ssh/known_hosts

VCノードに対して設定ファイルの変更やパッケージの追加を行う場合にVCノードの管理者権限が必要になる場合があります。Ansibleで管理者権限によるコマンド実行が可能かどうかを確認します。

In [ ]:
# 管理者権限(-b)でのコマンド実行
!ansible {target_group} -b -a 'whoami'

### group_vars ファイルの更新

In [ ]:
%run scripts/group.py

update_group_vars(
    target_group,
    _file='10-node.yml',
    aws_region=aws_region,
    instance_ids=instance_ids,
)

`group_vars`ファイルの内容を確認してみます。

In [ ]:
!cat group_vars/{target_group}/10-node.yml

## パッケージなどのインストール

Moodle環境を構築する際に必要となるパッケージなどのインストールを行います。

### Dockerのインストール

[Docker - Install Engine](https://docs.rockylinux.org/gemstones/containers/docker/) の手順に従い Docker のインストールを行います。

Dockerのレポジトリを追加します。

In [ ]:
!ansible {target_group} -b -a \
    'dnf config-manager --add-repo https://download.docker.com/linux/rhel/docker-ce.repo'

Dockerのパッケージをインストールします。

In [ ]:
!ansible {target_group} -b -m dnf -a 'name=docker-ce,docker-ce-cli,containerd.io,docker-buildx-plugin,docker-compose-plugin'

Docker Engine を実行するサービスを開始します。

In [ ]:
!ansible {target_group} -b -a 'systemctl start docker'

In [ ]:
!ansible {target_group} -b -a 'systemctl enable docker'

Docker Engine が実行されていることを確認するために `docker info` を実行してみます。

In [ ]:
!ansible {target_group} -b -a 'docker info'

dockerコマンドを管理者権限なしで実行できるようにするためにユーザを `docker` グループに所属させるようにします。

In [ ]:
!ansible {target_group} -b -m user -a 'name={{{{ansible_user}}}} append=yes groups=docker'

管理者権限なしで docker コマンドが実行できることを確認します。まず、新しいグループでログインし直すために、現在のsshの接続をいったん終了します。

In [ ]:
if os.path.exists(os.path.expanduser(f'~/.ansible/cp/rocky@{target_hostname}:22')):
    !ssh -o ControlPath=~/.ansible/cp/rocky@{target_hostname}:22 -O exit {target_hostname}

一般ユーザで `docker info`が実行できることを確認します。

In [ ]:
!ansible {target_group} -a 'docker info'

### Python3などのインストール

In [ ]:
!ansible {target_group} -b -m dnf -a 'name=python3.12,python3.12-pip,unzip,tree,git,wget,nmap-ncat,jq,which'

### aws cli のインストール

構築したMoodle環境をAmazon S3にバックアップする際に利用する aws cli のインストールを行います。

In [ ]:
!ansible {target_group} -m unarchive -a 'src=https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip \
    dest=/tmp remote_src=yes'
!ansible {target_group} -b -a 'chdir=/tmp/aws ./install'
!ansible {target_group} -a 'rm -rf /tmp/aws'

インストールされたことを確認するためにバージョンを表示してみます。

In [ ]:
!ansible {target_group} -a '/usr/local/bin/aws --version'

### タイムゾーンの変更

タイムゾーンをJSTに変更します。

In [ ]:
!ansible {target_group} -b -a 'timedatectl set-timezone Asia/Tokyo'

タイムゾーンが変更されたことを確認します。

In [ ]:
!ansible {target_group} -a 'date'